## AI Generated Topic Analysis

<b> Author:</b> Miraya Gupta \
<b> Date: </b> 04/06

In [107]:
import pandas as pd
import csv
import json
import numpy as np
import os
import io
import re
import string
import tqdm
import numpy as np
import matplotlib.pyplot as plt
#import tensorflow as tf
#from tensorflow.keras import layers
#from sklearn.feature_extraction.text import CountVectorizer

# Loading Files and Preprocessing

In [20]:
def get_df_for_year(year):
    '''
    Take in the year and return a df with data from that year
    '''
    directory = 'data'
    filename = f'data_w_topic_{year}.csv'
    path = os.path.join(directory, filename)
    df = pd.read_csv(path)
    return df

In [21]:
#calling function to get csv for each year
all_dfs = []
for yr in ['2020', '2021', '2023', '2024']:
    df = get_df_for_year(yr)
    all_dfs.append(df)

In [22]:
allDocs = pd.concat(all_dfs)

In [23]:
allDocs.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'video_id', 'video_timestamp',
       'video_duration', 'video_locationcreated', 'suggested_words',
       'video_diggcount', 'video_sharecount', 'video_commentcount',
       'video_playcount', 'video_description', 'video_is_ad', 'video_stickers',
       'author_username', 'author_name', 'author_followercount',
       'author_followingcount', 'author_heartcount', 'author_videocount',
       'author_diggcount', 'author_verified', 'search_term', 'year',
       'File Name', 'Content', 'Subjectivity/Objectivity', 'topic'],
      dtype='object')

In [24]:
allDocs['topic'].isnull().sum()

6

In [25]:
allDocs.shape

(11323, 28)

In [26]:
allDocs.dropna(subset=['topic'], inplace=True)

In [27]:
allDocs.shape

(11317, 28)

In [28]:
allDocs.drop(['Unnamed: 0.1', 'Unnamed: 0'], axis=1, inplace=True)

In [29]:
allDocs.shape

(11317, 26)

In [30]:
allDocs.set_index('video_id', inplace=True)

In [31]:
allDocs.index[:5]

Index([7104707494194269482, 7167954365922151726, 7277689387939417386,
       7176634327122365738, 7212690611852578094],
      dtype='int64', name='video_id')

In [37]:
allDocs = allDocs[allDocs['topic'] != 'not english']

In [38]:
allDocs.shape

(8882, 25)

In [62]:
allDocs = allDocs[allDocs['search_term'] != None]

In [63]:
allDocs = allDocs[allDocs['topic'] != None]

In [64]:
allDocs.shape

(8882, 25)

## Word2Vec Modelling

In [15]:
#from gensim.models import KeyedVectors
from gensim.models import Word2Vec
import gensim.downloader
word2vec_model = gensim.downloader.load('word2vec-google-news-300')
#model = Word2Vec.load('./model/GoogleNews-vectors-negative300.bin')
#print(embedding)

[-0.06445312 -0.16015625 -0.01208496  0.13476562 -0.22949219  0.16210938
  0.3046875  -0.1796875  -0.12109375  0.25390625 -0.01428223 -0.06396484
 -0.08056641 -0.05688477 -0.19628906  0.2890625  -0.05151367  0.14257812
 -0.10498047 -0.04736328 -0.34765625  0.35742188  0.265625    0.00188446
 -0.01586914  0.00195312 -0.35546875  0.22167969  0.05761719  0.15917969
  0.08691406 -0.0267334  -0.04785156  0.23925781 -0.05981445  0.0378418
  0.17382812 -0.41796875  0.2890625   0.32617188  0.02429199 -0.01647949
 -0.06494141 -0.08886719  0.07666016 -0.15136719  0.05249023 -0.04199219
 -0.05419922  0.00108337 -0.20117188  0.12304688  0.09228516  0.10449219
 -0.00408936 -0.04199219  0.01409912 -0.02111816 -0.13476562 -0.24316406
  0.16015625 -0.06689453 -0.08984375 -0.07177734 -0.00595093 -0.00482178
 -0.00089264 -0.30664062 -0.0625      0.07958984 -0.00909424 -0.04492188
  0.09960938 -0.33398438 -0.3984375   0.05541992 -0.06689453 -0.04467773
  0.11767578 -0.13964844 -0.26367188  0.17480469 -0.

In [65]:
row = allDocs.head(1)

In [66]:
row

,video_timestamp,video_duration,video_locationcreated,suggested_words,video_diggcount,video_sharecount,video_commentcount,video_playcount,video_description,video_is_ad,...,author_heartcount,author_videocount,author_diggcount,author_verified,search_term,year,File Name,Content,Subjectivity/Objectivity,topic
video_id,,,,,,,,,,,,,,,,,,,,,
7167954365922151726,2022-11-19T23:43:58,23.0,US,NaN,35800.0,974.0,228.0,323000.0,No public safety schedule is glamorous…!! #cop...,False,...,NaN,NaN,NaN,False,Public Safety,2020,@elevendouble9_video_7167954365922151726.txt,What's your schedule like coming here? We wor...,subjective,daily routine


In [87]:
def get_word_embedding(input_string):
    '''
    Get the concatenated word embedding for a topic and the search term. 
    '''
    try:
        words = input_string.split()
        embeddings = [word2vec_model[word] for word in words]
        if len(embeddings) == 1:
            return embeddings[0]
        concatenated_embedding = np.concatenate(embeddings)
        return concatenated_embedding
    except:
        return None

In [88]:
#from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial import distance

In [89]:
def compare_topic_search_term(search_emb, topic_emb):
    '''
    Comparing the word embeddings for search term and topic. 
    '''
    try:
        cosine_similarity = 1 - distance.cosine(search_emb, topic_emb)
        return cosine_similarity
    except:
        return None

In [90]:
def cosine_similarity(row):
    '''
    Function for df.apply()
    '''
    search_term = row['search_term'] #.values[0]
    topic = row['topic'] #.values[0]
    print(f'{search_term}, {topic}')
    search_emb = get_word_embedding(search_term)
    topic_emb = get_word_embedding(topic)
    return compare_topic_search_term(search_emb, topic_emb)

In [99]:
cosine_sim = []
for index, row in allDocs.iterrows():
    cosine_sim.append(cosine_similarity(row))

Public Safety, daily routine
Public Safety, theft
Public Safety, vaping
Public Safety, traffic cameras
Public Safety, parking
Public Safety, security policy
Public Safety, nurs
Public Safety, cartel
Public Safety, health bar
Public Safety, emergency response
Public Safety, daily routine
Public Safety, theft
Public Safety, vaping
Public Safety, traffic cameras
Public Safety, parking
Public Safety, security policy
Public Safety, nurs
Public Safety, cartel
Public Safety, health bar
Public Safety, public safety
Public Safety, daily routine
Public Safety, theft
Public Safety, vaping
Public Safety, traffic cameras
Public Safety, parking
Public Safety, security policy
Public Safety, nurs
Public Safety, cartel
Public Safety, health bar
Public Safety, public safety
Public Safety, daily routine
Public Safety, theft
Public Safety, traffic cameras
Public Safety, parking
Public Safety, security policy
Public Safety, nurs
Public Safety, drug trafficking
Public Safety, health bar
Public Safety, publi

In [93]:
cosine_sim[:10]

[0.06815919280052185,
 None,
 None,
 0.11162357032299042,
 None,
 0.07570777088403702,
 None,
 None,
 0.10849134624004364,
 0.09653569757938385]

In [96]:
none_vals = [val for val in cosine_sim if val == None]
print(len(none_vals))
print(len(cosine_sim))
print(1-(len(none_vals))/len(cosine_sim))

6503
8882
0.26784507993695117


In [98]:
allDocs['cosine_sim'] = cosine_sim

In [100]:
allDocs.head()

,video_timestamp,video_duration,video_locationcreated,suggested_words,video_diggcount,video_sharecount,video_commentcount,video_playcount,video_description,video_is_ad,...,author_videocount,author_diggcount,author_verified,search_term,year,File Name,Content,Subjectivity/Objectivity,topic,cosine_sim
video_id,,,,,,,,,,,,,,,,,,,,,
7167954365922151726,2022-11-19T23:43:58,23.0,US,NaN,35800.0,974.0,228.0,323000.0,No public safety schedule is glamorous…!! #cop...,False,...,NaN,NaN,False,Public Safety,2020,@elevendouble9_video_7167954365922151726.txt,What's your schedule like coming here? We wor...,subjective,daily routine,0.068159
7277689387939417386,2023-09-11T17:52:00,46.0,US,NaN,157600.0,2252.0,163.0,839100.0,That must mean the car is in the bike #college...,False,...,NaN,NaN,False,Public Safety,2020,@collegelifeshorts_video_7277689387939417386.txt,"Hey, I got my bike stolen. Oh nice. Okay. Gre...",subjective,theft,NaN
7176634327122365738,2022-12-13T09:06:40,20.0,US,"disposable straws, disposable pot",1100000.0,249900.0,13800.0,15800000.0,"@tiktok this ISN’T a promotion, its a PSA. (Bu...",False,...,NaN,NaN,False,Public Safety,2020,@adequatevapor_video_7176634327122365738.txt,I am reposting this here because I just assum...,subjective,vaping,NaN
7212690611852578094,2023-03-20T14:03:32,50.0,US,NaN,15700.0,2171.0,1110.0,299900.0,It’s not about “safety” and it never has been....,False,...,NaN,NaN,False,Public Safety,2020,@illinoispolicy_video_7212690611852578094.txt,"So this is the entrance to Horner Park, suppo...",subjective,traffic cameras,0.111624
7203203203419131178,2023-02-22T23:27:28,52.0,US,"safety, car parking, Parking For Disabled Ampu...",1251.0,21.0,33.0,61200.0,Man Confronts Cop Illegally Parked #publicsafe...,False,...,NaN,NaN,False,Public Safety,2020,@publicsafetyunleashed_video_72032032034191311...,I am so sorry to bother you here. You're park...,subjective,parking,NaN


In [101]:
allDocs.sort_values(by='cosine_sim', ascending=False, inplace=True)

In [106]:
allDocs = allDocs[allDocs['cosine_sim'] != None]

In [108]:
ft = allDocs['search_term'].value_counts()
ft

search_term
Political Debates                 142
Republican National Committee      97
Pence, Mike                        86
Harris, Kamala D                   80
Democratic National Committee      74
                                 ... 
National Conventions               12
Al Shifa Hospital (Gaza Strip)     10
Quds Force                          9
Sharon, Ariel                       9
Election Day                        8
Name: count, Length: 223, dtype: int64

In [ ]:
plt.barh(allDocs['search_term'][:10], allDocs['cosine'][:10])
plt.xlabel('Number of Documents Assigned to Topic')
plt.tick_params(axis='y', which='major', labelsize=8)
plt.savefig('docs_assigned_to_topic_top10.png', bbox_inches='tight')